In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from huggingface_hub import login
import re
import string
import time
import unicodedata

In [23]:
# open the txt file
with open("ffr_dataset_v2.txt", "r") as f:
    lines = f.readlines()
    for line in lines[:10]:
        print(line)


xota ðò akpà ɔ jí é etɛwu mǐ ðó nɔ ðɔ nugbǒ	couverture pourquoi être honnête

hitoshi nɔ ɖò tɔn ɖò nɔ nú mɛ é ɖé ɖò japon	hitoshi travaillait dans le comptabilité agence au japon

hwenu é ɖò akwɛzinzan ɖé xá gǎn tɔn wɛ é ɔ gǎn ɔ ɖɔ ɖɔ sɔgbe ǎ é ɖó nukún wɛ emi ɖè ɖò así tɔn	alors examinait avec responsable celui ci lui demandé le

hitoshi tinmɛ ɖɔ ayixa emitɔn kún yí gbè bonu emi wà byɔ nugbǒmaɖɔ é ó	hitoshi expliqué que lui permettait pas malhonnête

é sín enɛ wu bɔ gǎn ɔ adǎn ɖɔ emi nya ɛ sín bɔ ɖò gudo mɛ ɔ gblé dó è nugbǒ	il alors été menacé licenciement

ðò bɔ lɛ é mɛ ɔ awakanmɛ wá kú hitoshi bǐ dó é wà é biba wu	finalement il emploi

ði ɔ hwenu hitoshi ɖò wɛ gbè ɖokpo bɔ è ɖò nǔ kanbyɔ ɛ wɛ é ɔ é ɖɔ ɖɔ emi kún sixu wà nugbǒmaɖɔ é ó	dans les mois qui ont suivi hitoshi découragé il pensait jamais retrouver

mɛ ɖò nǔ kanbyɔ ɛ wɛ é ɖɔ ɖɔ linlin towe lɛ bú dò mɛ	entretien exemple après avoir précisé pouvait pas montrer malhonnête il entendu votre façon voir les choses est étrange

hɛn

In [25]:
# convert the txt file to a dataframe
data = []
with open("ffr_dataset_v2.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        # Split on the first tab occurrence only
        parts = line.strip().split("\t", 1)
        if len(parts) == 2:
            fon, french = parts
            data.append({"fon": fon.strip(), "french": french.strip()})
        else:
            # Handle lines that don't split correctly (optional)
            print(f"Skipping malformed line: {line}")

In [26]:
len(data)

54749

In [27]:
df = pd.DataFrame(data)

In [28]:
df

,fon,french
0,xota ðò akpà ɔ jí é etɛwu mǐ ðó nɔ ðɔ nugbǒ,couverture pourquoi être honnête
1,hitoshi nɔ ɖò tɔn ɖò nɔ nú mɛ é ɖé ɖò japon,hitoshi travaillait dans le comptabilité agenc...
2,hwenu é ɖò akwɛzinzan ɖé xá gǎn tɔn wɛ é ɔ gǎn...,alors examinait avec responsable celui ci lui ...
3,hitoshi tinmɛ ɖɔ ayixa emitɔn kún yí gbè bonu ...,hitoshi expliqué que lui permettait pas malhon...
4,é sín enɛ wu bɔ gǎn ɔ adǎn ɖɔ emi nya ɛ sín bɔ...,il alors été menacé licenciement
...,...,...
54744,Keze ɔ mɛ ni nyɔ nu mibi sese,Passez une belle journée
54745,Keze ɔ mɛ ni nyɔ nu wé sese,Que la journée te soit belle
54746,Keze ɔ mɛ ni nyɔ nu wé sese,Bonne journée
54747,Keze ɔ mɛ ni nyɔ nu wé sese,Passes une bonne journée


In [29]:
# remove duplicates
df = df.drop_duplicates(subset=["fon", "french"])
df

,fon,french
0,xota ðò akpà ɔ jí é etɛwu mǐ ðó nɔ ðɔ nugbǒ,couverture pourquoi être honnête
1,hitoshi nɔ ɖò tɔn ɖò nɔ nú mɛ é ɖé ɖò japon,hitoshi travaillait dans le comptabilité agenc...
2,hwenu é ɖò akwɛzinzan ɖé xá gǎn tɔn wɛ é ɔ gǎn...,alors examinait avec responsable celui ci lui ...
3,hitoshi tinmɛ ɖɔ ayixa emitɔn kún yí gbè bonu ...,hitoshi expliqué que lui permettait pas malhon...
4,é sín enɛ wu bɔ gǎn ɔ adǎn ɖɔ emi nya ɛ sín bɔ...,il alors été menacé licenciement
...,...,...
54744,Keze ɔ mɛ ni nyɔ nu mibi sese,Passez une belle journée
54745,Keze ɔ mɛ ni nyɔ nu wé sese,Que la journée te soit belle
54746,Keze ɔ mɛ ni nyɔ nu wé sese,Bonne journée
54747,Keze ɔ mɛ ni nyɔ nu wé sese,Passes une bonne journée


In [30]:
# remove rows with empty fon or french
df = df.dropna(subset=["fon", "french"])
df

,fon,french
0,xota ðò akpà ɔ jí é etɛwu mǐ ðó nɔ ðɔ nugbǒ,couverture pourquoi être honnête
1,hitoshi nɔ ɖò tɔn ɖò nɔ nú mɛ é ɖé ɖò japon,hitoshi travaillait dans le comptabilité agenc...
2,hwenu é ɖò akwɛzinzan ɖé xá gǎn tɔn wɛ é ɔ gǎn...,alors examinait avec responsable celui ci lui ...
3,hitoshi tinmɛ ɖɔ ayixa emitɔn kún yí gbè bonu ...,hitoshi expliqué que lui permettait pas malhon...
4,é sín enɛ wu bɔ gǎn ɔ adǎn ɖɔ emi nya ɛ sín bɔ...,il alors été menacé licenciement
...,...,...
54744,Keze ɔ mɛ ni nyɔ nu mibi sese,Passez une belle journée
54745,Keze ɔ mɛ ni nyɔ nu wé sese,Que la journée te soit belle
54746,Keze ɔ mɛ ni nyɔ nu wé sese,Bonne journée
54747,Keze ɔ mɛ ni nyɔ nu wé sese,Passes une bonne journée


In [35]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

def normalize_diacritics_text(text_string):
    """Convenience wrapper to abstract away unicode & NFC"""
    return unicodedata.normalize("NFC", text_string)

In [37]:
def preprocess_sentence(w):
    w = normalize_diacritics_text(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    w = re_punc.sub('', w)

    lines_str = w.replace("”", "")
    lines_str = lines_str.replace("“", "")
    lines_str = lines_str.replace("’", "'")
    lines_str = lines_str.replace("«", "")
    lines_str = lines_str.replace("»", "")
    lines_str = ' '.join([word for word in lines_str.split() if word.isalpha()])
    return lines_str


In [38]:
def preprocess_sentence_1(w):
    w = unicode_to_ascii(w.lower().strip())

    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    # w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    w = re_punc.sub('', w)

    lines_str = w.replace("”", "")
    lines_str = lines_str.replace("“", "")
    lines_str = lines_str.replace("’", "'")
    lines_str = lines_str.replace("«", "")
    lines_str = lines_str.replace("»", "")
    lines_str = ' '.join([word for word in lines_str.split() if word.isalpha()])
    return lines_str

In [39]:
df["fon"] = df["fon"].apply(preprocess_sentence)
df["french"] = df["french"].apply(preprocess_sentence_1)

In [42]:
df.head(100)

,fon,french
0,xota ðò akpà ɔ jí é etɛwu mǐ ðó nɔ ðɔ nugbǒ,couverture pourquoi etre honnete
1,hitoshi nɔ ɖò tɔn ɖò nɔ nú mɛ é ɖé ɖò japon,hitoshi travaillait dans le comptabilite agenc...
2,hwenu é ɖò akwɛzinzan ɖé xá gǎn tɔn wɛ é ɔ gǎn...,alors examinait avec responsable celui ci lui ...
3,hitoshi tinmɛ ɖɔ ayixa emitɔn kún yí gbè bonu ...,hitoshi explique que lui permettait pas malhon...
4,é sín enɛ wu bɔ gǎn ɔ adǎn ɖɔ emi nya ɛ sín bɔ...,il alors ete menace licenciement
...,...,...
95,mawu sín nyikɔ wɛ nɔ nyí jehovah ɖò biblu mɛ,jehovah est le nom dieu revele dans bible
96,biblu nɔ huzu gbɛzán mɛtɔn,bible transforme des vies
97,nyikɔ alain broggio,alain broggio
98,è jì mì ðò,annee


In [43]:
# Split the data into train, test, and validation sets
train, test = train_test_split(df, test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.2, random_state=42)

In [44]:
#  Delete the index column
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
val = val.reset_index(drop=True)

In [45]:
# Publish the datasets(train, test, val) to the huggingface hub
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)
val_dataset = Dataset.from_pandas(val)
dataset_dict = DatasetDict({"train": train_dataset, "test": test_dataset, "validation": val_dataset})
dataset_dict.push_to_hub("jonathansuru/fr_fon")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/36 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/854 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jonathansuru/fr_fon/commit/04d6346fb9bc122e0de2b0ff8973f050d6166ab0', commit_message='Upload dataset', commit_description='', oid='04d6346fb9bc122e0de2b0ff8973f050d6166ab0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/jonathansuru/fr_fon', endpoint='https://huggingface.co', repo_type='dataset', repo_id='jonathansuru/fr_fon'), pr_revision=None, pr_num=None)